<a href="https://colab.research.google.com/github/Prany25/OpenCV/blob/main/Simpsons_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install caer canaro

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.5/809.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 58.7 MB/s eta 0:00:00


In [4]:
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc
import tensorflow

In [ ]:
IMG_SIZE = (80,80)
channels = 1
char_path = r''   #data path yet to be uploaded

In [ ]:
char_dict = {}
for char in os.listdir(char_path):
  char_dict[char] = len(os.listdir(os.path.join(char_path,char)))

#sor this dictionary in descending order
char_dict = caer.sort_dict(char_dict, descending=True)
char_dict

In [ ]:
characters = []
count = 0
for i in char_dict:
  characters.append(i[0])
  count +=1
  if count >=10:
    break;
characters

In [ ]:
#create the training data
train = caer.preprocess_from_dir(char_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)

In [ ]:
from time import thread_time_ns
len(train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap='gray')
plt.show()

In [ ]:
featureSet, labels = caer.sep_train(train, IMG_size=IMG_SIZE)

In [ ]:
from tensorflow.keras.utils import to_categorical
#normalize the feature set ==> (0,1)
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [ ]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)

In [ ]:
del train
del featureSet
del labels
gc.collect()

In [ ]:
#image data generator
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size=32)

In [ ]:
#creating the model
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels, output_dim=len(characters),
loss='binary_crossentropy', decay=1e-6, learning_rate=0.001, momentum=0.9, nesterov=True)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
train = model.fit(train_gen, steps_per_epoch=len(x_train)//32, epochs=10, validation_data=(x_val, y_val), validation_steps=len(y_val)//32, callbacks=callbacks_list)

In [ ]:
characters

In [ ]:
test_path = r''  #add any file path
img = cv.imread(test_path)

plt.imshow(img)
plt.show()

In [ ]:
def prepare(img):
  img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
  img = cv.resize(img, IMG_SIZE)
  prepare_img = caer.reshape(img, IMG_SIZE, 1)
  return prepare_img

In [ ]:
predictions = model.predict(prepare(img))

In [ ]:
predictions

In [ ]:
print(characters[np.argmax(predictions[0])])